## Forecasting Groundwater Recharge

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import joblib

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

In [ ]:
df2 = pd.DataFrame()
n = [3, 4, 5, 12, 13, 14, 23, 24, 25, 44]
for i in range(1, 76):
    if i not in n:
        df = pd.read_excel("B:\GroundwaterRech\GithubRepo\Restart\Forecast_new\SSP126\District Wise\\" + str(i) + ".xlsx")
        df1 = df.drop(['Year', 'Longitude', 'Latitude'], axis=1)
        df2 = pd.concat([df2, df1])
        # df2 = pd.concat([df2, df])

In [ ]:
df2

In [ ]:
df2['Soil Type'] = df2['Soil Type'].replace('Builtup', 1)
df2['Soil Type'] = df2['Soil Type'].replace('Bulitup', 1)

df2['Soil Type'] = df2['Soil Type'].replace('Haplic Luvisols', 2)
df2['Soil Type'] = df2['Soil Type'].replace('Haplic Calcisols', 3)
df2['Soil Type'] = df2['Soil Type'].replace('Loam', 4)
df2['Soil Type'] = df2['Soil Type'].replace('Calcaric Fluvisols', 5)


##### Forecast

In [ ]:
X = df2

In [ ]:
X = pd.get_dummies(X)

In [ ]:
X

In [ ]:
scaler = joblib.load("B:\GroundwaterRech\GithubRepo\Restart\Results\Pretrained Model\scaler(Pre-Post).pkl")
X = scaler.transform(X)

In [ ]:
X

In [ ]:
m=joblib.load("B:\GroundwaterRech\GithubRepo\Restart\Results\Pretrained Model\Pre-post\XG Boost.pkl")
pred_array=np.array(m.predict(X))
pred_array.reshape(-1,16)

In [ ]:
m=joblib.load('XG Boost.pkl')
pred_array=np.array(m.predict(X))
pred_array.reshape(-1,16)

In [ ]:
std_predictions=pd.read_csv("B:\Out_53.csv")

In [ ]:
std_predictions=np.array(std_predictions['STD'])

Uncertaininty

In [ ]:
df3 = pd.DataFrame()
n = [3, 4, 5, 12, 13, 14, 23, 24, 25, 44]
for i in range(1, 76):
    if i not in n:
        df = pd.read_excel("B:\GroundwaterRech\GithubRepo\Restart\Dataset\District Wise\\2\\" + str(i) + ".xlsx")
        df1 = df.drop([ 'Year','Longitude', 'Latitude'], axis=1)
        df3 = pd.concat([df3, df1])

df3 = df3.reset_index()
df3.drop('index', inplace=True, axis=1)

df3['Soil Type'] = df3['Soil Type'].replace('Builtup', 1)
df3['Soil Type'] = df3['Soil Type'].replace('Bulitup', 1)

df3['Soil Type'] = df3['Soil Type'].replace('Haplic Luvisols', 2)
df3['Soil Type'] = df3['Soil Type'].replace('Haplic Calcisols', 3)
df3['Soil Type'] = df3['Soil Type'].replace('Loam', 4)
df3['Soil Type'] = df3['Soil Type'].replace('Calcaric Fluvisols', 5)
X1 = df3.drop('Groundwater Recharge', axis=1)
y1 = df3['Groundwater Recharge']

X1 = pd.get_dummies(X1)

scaler = MinMaxScaler()
X1 = scaler.fit_transform(X1)

In [ ]:
from sklearn.utils import resample
from xgboost import XGBRegressor

In [ ]:
n_bootstrap = 1000
bootstrap_predictions = np.zeros((n_bootstrap, len(X)))
for i in range(n_bootstrap):
    # Resample the training data with replacement
    X_boot, y_boot = resample(X1, y1, random_state=i)
    # Fit XGB model to the bootstrapped data
    xgb_boot = XGBRegressor(learning_rate=0.1, max_depth=8, n_estimators=150, random_state=i)
    xgb_boot.fit(X_boot, y_boot)

    # Make predictions on the test set
    bootstrap_predictions[i] = xgb_boot.predict(X)

In [ ]:
mean_predictions = np.mean(bootstrap_predictions, axis=0)
std_predictions = np.std(bootstrap_predictions, axis=0)
confidence_interval = np.percentile(bootstrap_predictions, [2.5, 97.5], axis=0)


In [ ]:
confidence_interval

In [ ]:
plt.figure(figsize=(10, 6))
plt.fill_between(range(len(X[:81])), confidence_interval[0][:81], confidence_interval[1][:81], color='gray', alpha=0.4, label='95% Confidence Interval')

# plt.plot(range(len(test_labels[:100])), test_labels[:100], label='True Values')
plt.plot(range(len(X[:81])), mean_predictions[:81], label='Mean Predictions', color='red')

plt.title('Groundwater Recharge Forecast with Uncertainty',fontsize=16,fontweight='bold')
plt.xlabel('Sample Index',fontsize=14,fontweight='bold')
plt.ylabel('Groundwater Recharge (m)',fontsize=14,fontweight='bold')
plt.legend()
plt.grid(True)
plt.savefig('B:\GroundwaterRech\GithubRepo\Restart\Results\Images\\uncertainty_forecast.png', dpi=1200)
plt.show()

In [ ]:
boots_p=bootstrap_predictions.transpose()

In [ ]:
boots_p